======================================================================================================

**Disclaimer**: Parts of this notebook are adapted from this [**Open Source Project**](https://github.com/AssistiveRoboticsUNH/bc_tutorial)  with minor modifications. Credit to the original authors.

======================================================================================================

# Workshop 7
## Behavior Cloning - Part 2 (Robotic Task)
=================================================================

### installation for colab
To Run in Colab: uncomment and run the following

In [ ]:
!git clone https://github.com/ARISE-Initiative/robomimic
!pip install -e robomimic/

import sys
import os
sys.path.append('./robomimic/')

# install all system dependencies for mujoco-py
!sudo apt install curl git libgl1-mesa-dev libgl1-mesa-glx libglew-dev \
         libosmesa6-dev software-properties-common net-tools unzip vim \
         virtualenv wget xserver-xorg-dev libglfw3-dev patchelf

#install mujoco-py
!pip install mujoco

#install robosuite
!pip install robosuite==1.4.1

# download lift ph dataset.
!wget http://downloads.cs.stanford.edu/downloads/rt_benchmark/lift/ph/low_dim_v141.hdf5 -O lift_ph_low_dim_v141.hdf5

Cloning into 'robomimic'...
remote: Enumerating objects: 4115, done.
remote: Counting objects: 100% (1903/1903), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 4115 (delta 1709), reused 1566 (delta 1566), pack-reused 2212 (from 1)
Receiving objects: 100% (4115/4115), 62.18 MiB | 25.82 MiB/s, done.
Resolving deltas: 100% (2950/2950), done.
Obtaining file:///content/robomimic
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.5/217.5 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
from copy import deepcopy
import datetime

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /content/./robomimic/robomimic/scripts/setup_macros.py
)


In [ ]:
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### Load Robomimic Lift Dataset

In [ ]:
dataset_path = "lift_ph_low_dim_v141.hdf5"
f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())
num_demos = len(demos)
print(f'Number of demos: {num_demos}')

Number of demos: 200


In [ ]:
select_keys=['object', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_gripper_qpos']

In [ ]:
# we will use a small dataset of first 20 demonstration for faster training
n_demo=20
actions_list=[]
obs_list=[]
for i in range(n_demo):
    # demo_id='demo_{}'.format(i)
    demo_id = demos[i]
    traj=f['data'][demo_id]

    actions=traj['actions']
    select_obs=np.hstack( [traj['obs'][key] for key in select_keys] )
    actions_list.append(actions)
    obs_list.append(select_obs)

actions_list=np.concatenate(actions_list)
obs_list=np.concatenate(obs_list)
obs_list.shape, actions_list.shape

((1001, 19), (1001, 7))

In [ ]:
data_loader = DataLoader( list(zip(obs_list, actions_list)), batch_size=64, shuffle=True)

batch=next(iter(data_loader))
states,actions = batch
states.shape,actions.shape

(torch.Size([64, 19]), torch.Size([64, 7]))

In [ ]:
action_dim=actions.shape[1]
state_dim=states.shape[1]
print(state_dim, action_dim)

19 7


### Model

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, size=32):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim,size),
            nn.ReLU(),
            nn.Linear(size,size),
            nn.ReLU()
        )
    def forward(self,x):
        x = self.net(x)
        return x

class RegNet(MLP):
    def __init__(self, input_dim , size, action_dim):
        super(RegNet, self).__init__(input_dim, size)
        self.decoder = nn.Linear(size, action_dim)
    def forward(self,x):
        x = self.net(x)
        x = self.decoder(x)
        return x

### Training

In [ ]:
learning_rate = 1e-4

bc = RegNet(state_dim, 64, action_dim)
criterion = nn.MSELoss()
optimizer = Adam(bc.parameters(), lr = learning_rate)

In [ ]:
loss_list = []
n_epoch = 3_000

for itr in range(0, n_epoch+1):
    total_loss = 0
    b=0
    for batch_states, batch_actions in data_loader:
        y_pred = bc(batch_states.float())
        loss   = criterion(y_pred, batch_actions.float())
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        b += 1

        loss_list.append(loss.item())

    if itr % (n_epoch//20)==0:
        print(f'Epoch {itr} Loss: {total_loss/b:.4f}')

Epoch 0 Loss: 0.1888
Epoch 150 Loss: 0.0627
Epoch 300 Loss: 0.0407
Epoch 450 Loss: 0.0342
Epoch 600 Loss: 0.0314
Epoch 750 Loss: 0.0292
Epoch 900 Loss: 0.0276
Epoch 1050 Loss: 0.0266
Epoch 1200 Loss: 0.0250
Epoch 1350 Loss: 0.0241
Epoch 1500 Loss: 0.0231
Epoch 1650 Loss: 0.0223
Epoch 1800 Loss: 0.0213
Epoch 1950 Loss: 0.0202
Epoch 2100 Loss: 0.0193
Epoch 2250 Loss: 0.0186
Epoch 2400 Loss: 0.0179
Epoch 2550 Loss: 0.0171
Epoch 2700 Loss: 0.0166
Epoch 2850 Loss: 0.0163
Epoch 3000 Loss: 0.0157


### Inference

In [ ]:
env_meta=FileUtils.get_env_metadata_from_dataset(dataset_path)
env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta,
    render=False,            # no on-screen rendering
    render_offscreen=True,   # off-screen rendering to support rendering video frames
)
dummy_spec = dict(  obs=dict( low_dim=["robot0_eef_pos"], rgb=[], ),)
ObsUtils.initialize_obs_utils_with_obs_specs(obs_modality_specs=dummy_spec)

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /usr/local/lib/python3.11/dist-packages/robosuite/scripts/setup_macros.py (macros.py:55)


Created environment with name Lift
Action size is 7

============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: []


In [ ]:
def rollout(env, rollout_horizon = 400, video_path=None):
    total_reward=0
    select_keys=['object', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_gripper_qpos']

    obs = env.reset()
    state_dict = env.get_state()
    # hack that is necessary for robosuite tasks for deterministic action playback
    # obs = env.reset_to(state_dict)

    if video_path is not None:
        video_writer = imageio.get_writer(video_path, fps=20)
        print(f"Writing video to {video_path}")
    camera_names=["agentview"]

    for step_i in range(rollout_horizon):
        select_obs=np.hstack( [obs[key] for key in select_keys] )
        state=torch.from_numpy(select_obs).float()
        # state=state.to(device='cuda')

        act = bc(state).detach().cpu().numpy()
        next_obs, r, done, _ = env.step(act)

        # compute reward
        total_reward += r
        success = env.is_success()["task"]

        if video_path is not None:
            video_img = env.render(mode="rgb_array", height=512, width=512, camera_name="agentview")
            video_writer.append_data(video_img)


        # break if done or if success
        if done or success:
            # print(f'stop: done={done} success={success}')
            break

        # update for next iter
        obs = deepcopy(next_obs)

    if video_path is not None:
        video_writer.close()
    return total_reward

In [ ]:
n_rollout=20
s=0
for i in range(n_rollout):
    r=rollout(env, video_path=None)
    s+=r
    print(f'Rollout {i} Success: {r}')

print(f'\nAverage Reward: {s/n_rollout:.2f}')

Rollout 0 Success: 0.0
Rollout 1 Success: 1.0
Rollout 2 Success: 1.0
Rollout 3 Success: 0.0
Rollout 4 Success: 1.0
Rollout 5 Success: 1.0
Rollout 6 Success: 1.0
Rollout 7 Success: 1.0
Rollout 8 Success: 1.0
Rollout 9 Success: 1.0
Rollout 10 Success: 1.0
Rollout 11 Success: 1.0
Rollout 12 Success: 1.0
Rollout 13 Success: 1.0
Rollout 14 Success: 1.0
Rollout 15 Success: 1.0
Rollout 16 Success: 1.0
Rollout 17 Success: 0.0
Rollout 18 Success: 1.0
Rollout 19 Success: 0.0

Average Reward: 0.80


### Render a video

In [ ]:
r=rollout(env, video_path="lift_bc.mp4")
r

Writing video to lift_bc.mp4


1.0

In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_path = 'lift_bc.mp4'

mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")